In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#create Spark session
spark = SparkSession.builder.appName('Stackoverflow_Project').getOrCreate()

#change configuration settings on Spark 
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '5g'), 
                                        ('spark.app.name', 'Spark Updated Conf'), 
                                        ('spark.executor.cores', '4'), 
                                        ('spark.cores.max', '4'), 
                                        ('spark.driver.memory','8g')])

In [4]:
# Define the path where the Parquet file is stored
input_path = "gs://msca-bdp-student-gcs/Group6/Tag_classification_inference/results_validation_data"

# Read the Parquet file into a PySpark DataFrame
results_validation_data_df_read = spark.read.parquet(input_path)

# Display the first few rows to verify the content
results_validation_data_df_read.show()

+--------------------+--------------------+----+---+----------+---+---+------+----+------+---+-------+---+---+-----+------+------+----------+------+---+-----+-----------+---------+----+-----------+--------+-----+-------------+-------+----+--------------+---+----+-------+----+----+----+--------+-------------+---+--------+----+--------+-----+----+---+-----+-----------------+-----------+---------------+----------+-------+
|      post_body_text|           post_tags|java| c#|javascript|php|c++|jquery|html|python|css|android|  c|sql|mysql|arrays|string|sql-server|iphone|ios|regex|objective-c|algorithm|ruby|performance|database|linux|ruby-on-rails|windows|list|multithreading|oop|bash|eclipse|ajax|perl|json|pointers|visual-studio|xml|winforms|linq|function|class|tsql|wpf|xcode|language-agnostic|asp_dot_net|asp_dot_net-mvc|vb_dot_net|dot_net|
+--------------------+--------------------+----+---+----------+---+---+------+----+------+---+-------+---+---+-----+------+------+----------+------+---+--

In [3]:
# Define the path where the Parquet file is stored
input_path = "gs://msca-bdp-student-gcs/Group6/Tag_classification_inference/results_validation_data_with_tags"

# Read the Parquet file into a PySpark DataFrame
results_validation_data_with_tags_df_read = spark.read.parquet(input_path)

# Display the first few rows to verify the content
results_validation_data_with_tags_df_read.show()

24/12/03 00:16:39 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+----+---+----------+---+---+------+----+------+---+-------+---+---+-----+------+------+----------+------+---+-----+-----------+---------+----+-----------+--------+-----+-------------+-------+----+--------------+---+----+-------+----+----+----+--------+-------------+---+--------+----+--------+-----+----+---+-----+-----------------+-----------+---------------+----------+-------+--------------------+
|      post_body_text|           post_tags|java| c#|javascript|php|c++|jquery|html|python|css|android|  c|sql|mysql|arrays|string|sql-server|iphone|ios|regex|objective-c|algorithm|ruby|performance|database|linux|ruby-on-rails|windows|list|multithreading|oop|bash|eclipse|ajax|perl|json|pointers|visual-studio|xml|winforms|linq|function|class|tsql|wpf|xcode|language-agnostic|asp_dot_net|asp_dot_net-mvc|vb_dot_net|dot_net|      predicted_tags|
+--------------------+--------------------+----+---+----------+---+---+------+----+------+---+-------+---+---+--

In [16]:
from pyspark.sql.functions import length, regexp_replace, col

results_validation_data_with_tags_df_read = results_validation_data_with_tags_df_read.withColumn(
    "tag_count", length(col("post_tags")) - length(regexp_replace(col("post_tags"), "\\|", "")) + 1)
results_validation_data_with_tags_df_read.show(5)

+--------------------+--------------------+----+---+----------+---+---+------+----+------+---+-------+---+---+-----+------+------+----------+------+---+-----+-----------+---------+----+-----------+--------+-----+-------------+-------+----+--------------+---+----+-------+----+----+----+--------+-------------+---+--------+----+--------+-----+----+---+-----+-----------------+-----------+---------------+----------+-------+--------------------+---------+
|      post_body_text|           post_tags|java| c#|javascript|php|c++|jquery|html|python|css|android|  c|sql|mysql|arrays|string|sql-server|iphone|ios|regex|objective-c|algorithm|ruby|performance|database|linux|ruby-on-rails|windows|list|multithreading|oop|bash|eclipse|ajax|perl|json|pointers|visual-studio|xml|winforms|linq|function|class|tsql|wpf|xcode|language-agnostic|asp_dot_net|asp_dot_net-mvc|vb_dot_net|dot_net|      predicted_tags|tag_count|
+--------------------+--------------------+----+---+----------+---+---+------+----+------+--

In [17]:
results_validation_data_with_tags_df_read.filter(size(col("predicted_tags")) == col("tag_count")).select('post_body_text','post_tags','predicted_tags').show(30)

+--------------------+--------------------+--------------------+
|      post_body_text|           post_tags|      predicted_tags|
+--------------------+--------------------+--------------------+
| i am trying to s...|             android|           [android]|
| i have a table a...|sql|sql-server|pe...|[sql, mysql, sql-...|
| i have code that...|javascript|jquery...|[javascript, jque...|
| good afternoon i...|                  c#|               [c++]|
| i have a piece o...|c#|memory|optimiz...|     [java, c#, c++]|
| i m in a program...|            c|arrays|         [c, arrays]|
| i have an array ...|                ruby|            [string]|
| i have two group...|                java|              [java]|
| i would like to ...|sql|sql-server-20...|[python, sql, sql...|
| how can i close ...|c#|winforms|keybo...|[c#, string, winf...|
| i m confused ans...|  c#|synchronization|[java, multithrea...|
| i need round cor...|html|css|rounded-...|[javascript, jque...|
| i trying to crea...|   

In [18]:
filtered = results_validation_data_with_tags_df_read.filter(size(col("predicted_tags")) == col("tag_count")).select('post_body_text','post_tags','predicted_tags')
filtered.count()

257

In [21]:
filtered1 = filtered.toPandas()
filtered1.to_csv('filtered.csv',index=False)

### Get sample posts to match with stackoverflow website post

In [4]:
#Read the cleaned and pre-processed data from the GCS bucket
df = spark.read \
    .option("quote", "\"")  \
    .option("escape", "\"") \
    .option("ignoreLeadingWhiteSpace",True) \
    .parquet("gs://msca-bdp-student-gcs/Group6/extracted_StackOverflow.parquet",inferSchema=True, header=True )

In [6]:
from pyspark.sql.functions import split, explode, col, lit, array_contains

df_1 = df.filter(col('post_tags')=='javascript|jquery|html|css')

In [7]:
df_1.show(5)

+--------+--------------------+--------------------+-----------------------+-----------------+------------------+------------------+-------------------+-----------------------+-------------------+------------------------+------------------+-----------------+----------+--------------------+---------------+---------+--------------------+--------------------+--------------------+-------------------------+---------------------+--------------------------+--------------------+----------------+-------------------+------------+--------------------+--------------------+
| post_id|          post_title|           post_body|post_accepted_answer_id|post_answer_count|post_comment_count|post_creation_date|post_favorite_count|post_last_activity_date|post_last_edit_date|post_last_editor_user_id|post_owner_user_id|post_post_type_id|post_score|           post_tags|post_view_count|answer_id|         answer_body|answer_comment_count|answer_creation_date|answer_last_activity_date|answer_last_edit_date|answe

In [8]:
df_1.count()

5750

In [16]:
df_2 = df.filter(col('post_body_text')==''' i m a trying to find a sublist of a list meaning if list1 say 15 is in list2 say 14356 than it should return true what i have so far is this this would be true but i m trying to return true only if list1 is in list2 in the respective order so if list2 is 52341 it should return false i was thinking along the lines of comparing the index values of list1 using lt but i m not sure ''')

In [17]:
df_2.select('post_title').show(truncate=False)

+-------------------------------------------------------+
|post_title                                             |
+-------------------------------------------------------+
|Searching values of a list in another List using Python|
|Searching values of a list in another List using Python|
|Searching values of a list in another List using Python|
|Searching values of a list in another List using Python|
|Searching values of a list in another List using Python|
|Searching values of a list in another List using Python|
|Searching values of a list in another List using Python|
+-------------------------------------------------------+



In [18]:
df_2.count()

7